In [1]:
"""
Coin recognition, real life application
task: calculate the value of coins on picture
Currently, image must contain a 1 cent coin in order to function
"""

import cv2
import numpy as np

#This is a function used for the detection of coins
def detect_coins():
    #Read in the image and convert it to grayscale
    coins = cv2.imread('Test_coins_1.jpg', 1)
    gray = cv2.cvtColor(coins, cv2.COLOR_BGR2GRAY)
    
    #This adds a bit of blur to th image to reduce noise, making it easier to detect circles
    #We then detect circles using the HoughCircles algorithm
    img = cv2.medianBlur(gray, 7)
    circles = cv2.HoughCircles(
        img,  # source image
        cv2.HOUGH_GRADIENT,  # type of detection
        1,
        50,
        param1=115,
        param2=100,
        minRadius=40,  # minimal radius
        maxRadius=500,  # max radius
    )
    
    #Create a copy of the coins image
    coins_copy = coins.copy()

    #Draw each of the circles(coins) that were detected within the image onto the image
    for detected_circle in circles[0]:
        x_coor, y_coor, detected_radius = detected_circle
        coins_detected = cv2.circle(
            coins_copy,
            (int(x_coor), int(y_coor)),
            int(detected_radius),
            (0, 255, 0),
            4,
        )
    
    #create the image and then return the list of circles 
    cv2.imwrite("Circled_coins.jpg", coins_detected)
    return circles

#Function for calculating the amount each coin is worth
def calculate_amount():
    #This is an array holding information on each coin. This includes it's average radius,
    #and it's ratio of size when compared to a 1 cent coin, and the value of the coin
    coins = {
        "0.01 EUR": {
            "value": 1,
            "radius": 16.25,
            "ratio": 1,
            "count": 0,
        },
        "0.02 EUR": {
            "value": 2,
            "radius": 18.75,
            "ratio": 1.154,
            "count": 0,
        },
        "0.05 EUR": {
            "value": 5,
            "radius": 21.25,
            "ratio": 1.308,
            "count": 0,
        },
        "0.1 EUR": {
            "value": 10,
            "radius": 19.75,
            "ratio": 1.215,
            "count": 0,
        },
        "0.2 EUR": {
            "value": 20,
            "radius": 22.25,
            "ratio": 1.369,
            "count": 0,
        },
        "0.5 EUR": {
            "value": 50,
            "radius": 24.25,
            "ratio": 1.492,
            "count": 0,
        },
        "1 EUR": {
            "value": 100,
            "radius": 23.25,
            "ratio": 1.431,
            "count": 0,
        },
        "2 EUR": {
            "value": 200,
            "radius": 25.75,
            "ratio": 1.585,
            "count": 0,
        },
    }
    
    #This calls the function to detect all coins, and returns all circles found
    circles = detect_coins()
    radius = []
    coordinates = []
    
    #This adds the co-ordinates and radius of each circle to an array
    for detected_circle in circles[0]:
        x_coor, y_coor, detected_radius = detected_circle
        radius.append(detected_radius)
        coordinates.append([x_coor, y_coor])
    
    #This finds the smallest radius within all of the detected circles, and assumes that coins is 1 cent
    smallest = min(radius)
    tolerance = 0.03
    total_amount = 0
    
    #This creates the image that the circles will be drawn on and sets the font that it will be written in
    coins_circled = cv2.imread('Circled_coins.jpg', 1)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #This loops through all coins that were found
    for coin in circles[0]:
        #This finds the ratio of the current coin we are working on when compared to a 1 cent coin
        ratio_to_check = coin[2] / smallest
        #Finds the x and y co-ordinate of this circle
        coor_x = coin[0]
        coor_y = coin[1]
        #This will loop this coin through all coins within our detailed coin array
        for coin in coins:
            #This gets the value of the current coin we are working with
            value = coins[coin]['value']
            #This is a check to see if the ratio of the circle we have to the smallest circle is within the
            #tolerance of the current coin we are checking within the list. If so, we add the value of the coin
            #to the total amount, amnd then add that value onto the coin within the image
            if abs(ratio_to_check - coins[coin]['ratio']) <= tolerance:
                coins[coin]['count'] += 1
                total_amount += coins[coin]['value']
                cv2.putText(coins_circled, str(value/100), (int(coor_x), int(coor_y)), font, 1,
                            (0, 0, 0), 4)

    #This outputs the total amount detected within the image
    print(f"The total amount is {float(total_amount/100.00)} EUR")
    #This outputs the total amount of each coin detected
    for coin in coins:
        pieces = coins[coin]['count']
        print(f"{coin} = {pieces}")


    #This writes the output image
    cv2.imwrite("Circled_coins.jpg", coins_circled)


#This calls the function to calculate the amount within the  image
if __name__ == "__main__":
    calculate_amount()

The total amount is 3.88 EUR
0.01 EUR = 1
0.02 EUR = 1
0.05 EUR = 1
0.1 EUR = 1
0.2 EUR = 1
0.5 EUR = 1
1 EUR = 1
2 EUR = 1
